# 多层感知机：Multi-Layer Perceptron



In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE' # plt绘图报错（不显示图的问题）
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [2]:
mnist_train = torchvision.datasets.FashionMNIST(root='F:/bigdata/ai/Pytorch_Datasets', train=True, download=False, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='F:/bigdata/ai/Pytorch_Datasets', train=False, download=False, transform=transforms.ToTensor())

batch_size = 256

train_dataloader = DataLoader(mnist_train, batch_size=batch_size)
test_dataloader = DataLoader(mnist_test, batch_size=batch_size)

In [3]:
# 计算准确率
def evaluate_accuracy(test_data_iter, model):
    acc_sum, n = 0.0, 0
    for X, y in test_data_iter:
        acc_sum += (model(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [4]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

class MultiLayerPerceptron(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(MultiLayerPerceptron, self).__init__()
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_inputs, num_hiddens),
            nn.ReLU(),
            nn.Linear(num_hiddens, num_outputs),
        )
    def forward(self, x): # x shape: (batch, 1, 28, 28)
        # y = self.linear(x.view(x.shape[0], -1))
        y = self.linear(x)
        # y = torch.softmax(y,dim=1) #不需要，加了效果反而不好
        return y

model = MultiLayerPerceptron(num_inputs, num_outputs)

# for params in model.parameters():
#     init.normal_(params, mean=0, std=0.01)

# nn.CrossEntropyLoss()中的label不需要是one_hot。要求是一维的label。
# 在使用该函数前 input不需要经过softmax计算， target不是one_hot编码格式
loss = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [5]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    model.train()
    for X, y in train_dataloader:
        # 计算预测误差
        output = model(X)
        # y = torch.nn.functional.one_hot(y,10) # 不需要
        l = loss(output, y)

        # 反向传播
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    model.eval()
    print('epoch %d, loss: %f, acc: %f' % (epoch, l.item() , evaluate_accuracy(test_dataloader,model)))

epoch 1, loss: 1.260179, acc: 0.645000
epoch 2, loss: 0.900895, acc: 0.679800
epoch 3, loss: 0.762826, acc: 0.714800
epoch 4, loss: 0.684754, acc: 0.739200
epoch 5, loss: 0.631079, acc: 0.755300
epoch 6, loss: 0.591212, acc: 0.770300
epoch 7, loss: 0.560809, acc: 0.782400
epoch 8, loss: 0.537280, acc: 0.789000
epoch 9, loss: 0.518899, acc: 0.793900
epoch 10, loss: 0.504275, acc: 0.798000


In [6]:
for i in range(20):
    #print(np.argmax(model(mnist_test[i][0]).detach().numpy()) ,"---实际：", mnist_test[i][1] )
    print(model(mnist_test[i][0]).argmax(dim=1) ,"---实际：", mnist_test[i][1] )

tensor([9]) ---实际： 9
tensor([2]) ---实际： 2
tensor([1]) ---实际： 1
tensor([1]) ---实际： 1
tensor([6]) ---实际： 6
tensor([1]) ---实际： 1
tensor([4]) ---实际： 4
tensor([6]) ---实际： 6
tensor([5]) ---实际： 5
tensor([7]) ---实际： 7
tensor([4]) ---实际： 4
tensor([5]) ---实际： 5
tensor([5]) ---实际： 7
tensor([3]) ---实际： 3
tensor([4]) ---实际： 4
tensor([1]) ---实际： 1
tensor([2]) ---实际： 2
tensor([6]) ---实际： 4
tensor([8]) ---实际： 8
tensor([0]) ---实际： 0
